<img src="https://www.th-koeln.de/img/logo.svg" style="float:right;" width="200">

# 13th exercise: <font color="#C70039">First Reinforcement Learning Q-Table learning</font>
* Course: AML
* Lecturer: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook: Finn Heydemann
* Date:   14.01.2024

---------------------------------
**GENERAL NOTE 1**: 
Please make sure you are reading the entire notebook, since it contains a lot of information on your tasks (e.g. regarding the set of certain paramaters or a specific computational trick), and the written mark downs as well as comments contain a lot of information on how things work together as a whole. 

**GENERAL NOTE 2**: 
* Please, when commenting source code, just use English language only. 
* When describing an observation please use English language, too.
* This applies to all exercises throughout this course.

---------------------------------

### <font color="FFC300">TASKS</font>:
The tasks that you need to work on within this notebook are always indicated below as bullet points. 
If a task is more challenging and consists of several steps, this is indicated as well. 
Make sure you have worked down the task list and commented your doings. 
This should be done by using markdown.<br> 
<font color=red>Make sure you don't forget to specify your name and your matriculation number in the notebook.</font>

**YOUR TASKS in this exercise are as follows**:
1. import the notebook to Google Colab or use your local machine.
2. make sure you specified you name and your matriculation number in the header below my name and date. 
    * set the date too and remove mine.
3. read the entire notebook carefully 
    * add comments whereever you feel it necessary for better understanding
    * run the notebook for the first time. 
4. play with all hypterparameters including the actions and states table.
5. add and implement an ϵ-greedy strategy 
---------------------------------

In [1]:
# Imports 
import numpy as np

### Create the possible states

In [6]:
location_to_state = {
    'L1' : 0,
    'L2' : 1,
    'L3' : 2,
    'L4' : 3,
    'L5' : 4,
    'L6' : 5,
    'L7' : 6,
    'L8' : 7,
    'L9' : 8
}

state_to_location = dict((state,location) for location, state in location_to_state.items())
state_to_location

{0: 'L1',
 1: 'L2',
 2: 'L3',
 3: 'L4',
 4: 'L5',
 5: 'L6',
 6: 'L7',
 7: 'L8',
 8: 'L9'}

### Create the actions & rewards

In [6]:
actions = [0,1,2,3,4,5,6,7,8]

In [7]:
rewards = np.array([[0,1,0,0,0,0,0,0,0],
                   [1,0,1,0,1,0,0,0,0],
                   [0,1,0,0,0,1,0,0,0],
                   [0,0,0,0,0,0,1,0,0],
                   [0,1,0,0,0,0,0,1,0],
                   [0,0,1,0,0,0,0,0,0],
                   [0,0,0,1,0,0,0,1,0],
                   [0,0,0,0,1,0,1,0,1],
                   [0,0,0,0,0,0,0,1,0]])

### Def remaining hyperparameters

In [8]:
# initialize the parameters
gamma = 0.75 # discount factor
alpha = 0.9  # learning rate

### Define agent and its attributes

In [9]:
class QAgent():
    # initialize everything
    def __init__(self, alpha, gamma, location_to_state, actions, rewards, state_to_location):
        self.gamma = gamma
        self.alpha = alpha
        
        self.location_to_state = location_to_state
        self.actions = actions
        self.rewards = rewards
        self.state_to_location = state_to_location
        
        # remember, the Q-value table is of size all actions x all states
        M = len(location_to_state)
        self.Q = np.zeros((M,M), dtype = None, order = 'C')
        
    # now, implement the training method for the agent
    def training(self, start_location, end_location, iterations):

        rewards_new = np.copy(self.rewards)

        ending_state = self.location_to_state[end_location]
        
        rewards_new[ending_state, ending_state] = 999
        
        # DEBUG
        print(rewards_new)

        # pick random current state
        # iterations = the # of training cycles
        for i in range(iterations):
            current_state = np.random.randint(0,9)
            playable_actions = []

            # iterate thru the rewards matrix to get states
            # that are really reachable from the randomly chosen
            # state and assign only those in a list since they are really playable
            for j in range(9):
                if rewards_new[current_state, j] > 0:
                    playable_actions.append(j)

            # choosing next random state
            # however, make sure that playable_actions is not empty
            if len(playable_actions) != 0:
                # Randomly select a value from playable_actions
                next_state = np.random.choice(playable_actions)
    
            # finding the difference in Q, often referred to as temporal difference
            # by means of the Bellman's equation
            TD = rewards_new[current_state, next_state] + self.gamma * self.Q[next_state, np.argmax(self.Q[next_state,])] - self.Q[current_state, next_state]
            # combine with the learning rate
            self.Q[current_state, next_state] += self.alpha*TD
            # DEBUG
            #print(f"Q[{current_state}, {next_state}]:", self.Q[current_state, next_state])

        route = [start_location]
        next_location = start_location

        # print the optimal route from start to end
        self.get_optimal_route(start_location, end_location, next_location, route, self.Q)

    # compute the optimal route
    def get_optimal_route(self, start_location, end_location, next_location, route, Q):
        while(next_location != end_location):
            starting_state = self.location_to_state[start_location]
            next_state = np.argmax(Q[starting_state,])
            next_location = self.state_to_location[next_state]
            route.append(next_location)
            start_location = next_location
        # DEBUG
        print('Q-table:',Q)
        print("optimal route:", route)

In [10]:
qagent = QAgent(alpha, gamma, location_to_state, actions, rewards, state_to_location)
qagent.training('L9', 'L1', 1000)

[[999   1   0   0   0   0   0   0   0]
 [  1   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0   0   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Q-table: [[3995.99457641 2249.49593613    0.            0.            0.
     0.            0.            0.            0.        ]
 [2997.99473438    0.         1688.12087724    0.         1688.12193195
     0.            0.            0.            0.        ]
 [   0.         2249.4960494     0.            0.            0.
  1267.0900769     0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.          951.31706967    0.            0.        ]
 [   0.         2249.49593487    0.            0.            0.
     0.            0.         1267.08884021    0.        ]
 [   0.         